# Chapter 14: Errors and exceptions

Sometimes, things don't go according to plan. To indicate that something exceptional has happened, Python provides exceptions. 

We indicate that something bad has happened with the raise statement:

In [ ]:
def raiseHell(doIt):
    if doIt:
        raise OverflowError("something bad!")
    else:
        return 1/0

Try running both *raiseHell*(True) and *raiseHell*(False).

You can see that some of the text is different, but it's the same mechanism. 

Here are some names of exception types, use whatever seems appropriate.

(There are also mechanisms to define your own, see online documentation.)

In [ ]:
# Exception
# |
# |--ArithmeticError
# |  |
# |  |--OverflowError
# |  |
# |  |--ZeroDivisionError
# |  |
# |  |--FloatingPointError
# |
# |--LookupError
# |  |
# |  |--IndexError
# |  |
# |  |--KeyError
# |
# |--NameError
# |
# |--StopIteration
# |
# |--TypeError
# |
# |--ValueError
# |

Okay, now to discuss how to catch exceptions (Once I get through this, the shape of that graph will make a lot more sense.)

You indicate that you think a block of code may throw an exception by putting it in a try block. 

If an exception is raised, you can stop it with the except keyword, followed by the type of exception you want to deal with. Then, put in the appropriate error recovery code. 

In [ ]:
def calmDown(doIt):
    try:
        raiseHell(doIt)
    except (OverflowError):
        print("Caught an exception, calming down.")
    return 0

Run this twice: Once with *doIt* set to True, once as False.

Notice that when *raiseHell* raises a *ZeroDivisionError*, the except block doesn't deal with it. It only looks for *OverflowError*, and 1/0 is not an *OverflowError*.

Exceptions propagate up a function call stack:

In [ ]:
def exceptCallStack():
    def a():
        raise OverflowError("Exception raised in a")
    def b():
        try:
            a()
        except(KeyError):
            print("Caught a key error.")
    def c():
        try:
            b()
        except(OverflowError):
            print("Caught an OverflowError in c")
    def d():
        try:
            c()
        except(OverflowError):
            print("Caught an OverflowError in d")

    d()
#Note that d does not catch an OverflowError - it has already been handled by c.

Okay, now that funky shape of the exception list. Exceptions have a heierarchy.

All exception types (*OverflowError*, *KeyError*, etc.) are a subset of *Exception*.

This means that if you

In [2]:
#except (Exception):

You will catch all exception types.

If you catch *ArithmeticError*, you will catch *OverflowError* and *ZeroDivisionError*, but a *KeyError* will pass on up.

Okay, what happens if you get an exception and the code that caused it wasn't inside a try block?

In [ ]:
def noTry():
    def a():
        print("entered A")
        raise OverflowError("A's exception")
        print("Finishing A.")
    def b():
        print("Entering B.")
        a()
        print("Finished B")
    def c():
        print("entering C")
        try:
            b()
        except(OverflowError):
            print("caught B's exception in C")
        print("leaving C")
    def d():
        print("entering D.")
        c()
        print("leaving D")
    d()


So essentially, when there's an exception flying around, you go directly
to jail, do not pass go, do not collect 200 dollars. You jump up the call
stack and skip all the code you come across until you find an except block.

Okay, last thing about exceptions. This "do not pass go" behavior of
exceptions can be very bad... (Don't run this function, as it refers to undefined code)

In [ ]:
def stupidFile(fname): # DO NOT RUN **
    f = open(fname, mode = 'r')
    performRiskyOperation(f)
    f.close()

If everything goes according to plan, you open the file, do something, then close it back down. But if performRiskyOperation barfs, you skip the *f.close*() line, and this means other parts of your program may not be able to access the file, or something else horrible may happen.

But, *stupidFile*() doesn't really know how to handle the exception that might be raised, so it would be unwise to just swallow it:

In [ ]:
def silentFail(fname):
    f = open(fname, mode = 'r')
    try:
        performRiskyOperation(f)
    except(Exception):
        pass
    f.close()

This is guaranteed to close the file, but doesn't tell the caller that something bad happened. What we really should do is close the file, then re-raise the exception. Inside an *except* block, the *raise* keyword alone causes the current exception to happen again:

In [ ]:
def safeFail(fname):
    f = open(fname, mode = 'r')
    try:
        performRiskyOperation(f)
    except(Exception):
        f.close()
        raise
    f.close()

This pattern is so common, that there's another keyword for it: the *finally* keyword. the finally block executes whether or not an exception occured in a *try* block, but doesn't shush the exception.

In [ ]:
def finallyFail():
    f = open(fname, mode = 'r')
    try:
        performRiskyOperation(f)
    finally:
        f.close()

####   *Exercises*

1 - List a few situations where an exception may arise that are relevant to code you've written for this series. 

2 - Choose a function you've written before and add error checking to it, raising an appropriate exception if something goes wrong.

3 - *float*(x) is a built-in function. It converts a string to a floating-point number, or raises an exception if this is not possible.

Write a function that checks to see if a string can be converted to a number; it should return *True* or *False*.   

In [ ]:
def isNumeric(x):
    pass